In [1]:
import tensorflow as tf
import string
import requests

In [15]:
response=requests.get("https://archive.org/details/isbn_9783822889633") 


In [16]:
response.text[:1500] 

'<!DOCTYPE html>\n<html lang="en">\n<!-- __ _ _ _ __| |_ (_)__ _____\n    / _` | \'_/ _| \' \\| |\\ V / -_)\n    \\__,_|_| \\__|_||_|_| \\_/\\___| -->\n  <head data-release=ed6cd0a4>\n    <title>Erotica universalis : Néret, Gilles : Free Download, Borrow, and Streaming : Internet Archive</title>\n\n          <meta name="viewport" content="width=device-width, initial-scale=1.0"/>\n    \n        <meta name="google-site-verification" content="Q2YSouphkkgHkFNP7FgAkc4TmBs1Gmag3uGNndb53B8" />\n    <meta name="google-site-verification" content="bpjKvUvsX0lxfmjg19TLblckWkDpnptZEYsBntApxUk" />\n\n    <script>\n/* @licstart  The following is the entire license notice for the\n * JavaScript code in this page.\n *\n * This program is free software: you can redistribute it and/or modify\n * it under the terms of the GNU Affero General Public License as published by\n * the Free Software Foundation, either version 3 of the License, or\n * (at your option) any later version.\n *\n * This program is 

In [17]:
data = response.text.split('\n') 
data[0] 

'<!DOCTYPE html>'

In [19]:
data = data[253:] 
data[0]

'      }'

In [20]:
len(data) 

2071

In [21]:
data = " ".join(data) 
data[:1000] 

'      }        .media-menu-1 .media-menu.tx-slide.open.media-menu {         top: 100%;       }        .media-menu-1 .media-menu.tx-slide.closed.media-menu {         top: -100vh;       }        .media-menu-1 .media-menu.tx-slide.closed.media-menu {         transition-duration: 0.2s;       }        .media-menu-1 .menu-group.media-menu {         position: relative;           line-height: normal;       }        @media (min-width: 890px) {       .media-menu-1 .media-menu.media-menu {         display: inline-block;             position: static;             width: auto;             height: 5rem;       }        .media-menu-1 .media-menu.tx-slide.media-menu {         transition-property: none;       }        .media-menu-1 .media-menu.tx-slide.open.media-menu,.media-menu-1 .media-menu.tx-slide.closed.media-menu {         top: 0;       }        .media-menu-1 .menu-group.media-menu {         font-size: 0;       }        }</style><!-- Shady DOM styles for primary-nav --><style scope="primary-nav-1

In [22]:
def clean_text(doc): 
 tokens = doc.split() 
 table = str.maketrans('', '', string.punctuation) 
 tokens = [w.translate(table) for w in tokens] 
 tokens = [word for word in tokens if word.isalpha()] 
 tokens = [word.lower() for word in tokens] 
 return tokens 
tokens = clean_text(data) 
print(tokens[:50]) 


['mediamenutxslideopenmediamenu', 'top', 'mediamenutxslideclosedmediamenu', 'top', 'mediamenutxslideclosedmediamenu', 'transitionduration', 'menugroupmediamenu', 'position', 'relative', 'lineheight', 'normal', 'media', 'minwidth', 'mediamenumediamenu', 'display', 'inlineblock', 'position', 'static', 'width', 'auto', 'height', 'mediamenutxslidemediamenu', 'transitionproperty', 'none', 'mediamenutxslideclosedmediamenu', 'top', 'menugroupmediamenu', 'fontsize', 'style', 'shady', 'dom', 'styles', 'for', 'primarynav', 'style', 'inputprimarynavfocus', 'outline', 'none', 'navprimarynav', 'position', 'relative', 'display', 'msgrid', 'display', 'grid', 'height', 'gridtemplateareas', 'hamburger', 'empty', 'search']


In [23]:
len(tokens) 

3234

In [27]:
length = 50 + 1 
lines = [] 
for i in range(length, len(tokens)): 
 seq = tokens[i-length:i] 
 line = ' '.join(seq) 
 lines.append(line) 
 if i > 200000: 
    break 
print(len(lines)) 

3183


#Build LTSM Model

In [29]:
#BUILD LTSM MODEL
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [32]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(lines) 
sequences = tokenizer.texts_to_sequences(lines) 

In [33]:
sequences = np.array(sequences) 
X, y = sequences[:, :-1], sequences[:,-1] 
X[0] 

array([1100,   19, 1083,   19, 1083,   92, 1081,   32,  107, 1097, 1096,
         61,   68, 1094,    8,  108,   32,  328,   67,   36,   41, 1089,
        194,    9, 1083,   19, 1081,   31,   37,   60,   59,   58,   33,
         16,   37, 1079, 1078,    9,  326,   32,  107,    8,  193,    8,
        192,   41, 1073,  323, 1072,   90])

In [57]:
vocab_size = len(tokenizer.word_index) + 1

In [36]:
y = to_categorical(y, num_classes=vocab_size) 

In [37]:
seq_length = X.shape[1] 
seq_length 


50

In [39]:
# LTSM MODEL

model = Sequential() 
model.add(Embedding(vocab_size, 50, input_length=seq_length)) 
model.add(LSTM(100, return_sequences=True)) 
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu')) 
model.add(Dense(vocab_size, activation='softmax')) 

In [40]:
model.summary() 


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            55100     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1102)              111302    
Total params: 317,302
Trainable params: 317,302
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 

In [42]:
model.fit(X, y, batch_size = 256, epochs = 100) 

Epoch 1/100
13/13 [==============================] - 5s 203ms/step - loss: 6.9918 - accuracy: 0.0258
Epoch 2/100
13/13 [==============================] - 3s 191ms/step - loss: 6.4971 - accuracy: 0.0145
Epoch 3/100
13/13 [==============================] - 3s 211ms/step - loss: 6.1078 - accuracy: 0.0368
Epoch 4/100
13/13 [==============================] - 2s 189ms/step - loss: 6.0536 - accuracy: 0.0377
Epoch 5/100
13/13 [==============================] - 3s 192ms/step - loss: 6.0355 - accuracy: 0.0390
Epoch 6/100
13/13 [==============================] - 3s 200ms/step - loss: 6.0280 - accuracy: 0.0415
Epoch 7/100
13/13 [==============================] - 3s 211ms/step - loss: 6.0254 - accuracy: 0.0415
Epoch 8/100
13/13 [==============================] - 3s 226ms/step - loss: 6.0237 - accuracy: 0.0415
Epoch 9/100
13/13 [==============================] - 3s 246ms/step - loss: 6.0236 - accuracy: 0.0415
Epoch 10/100
13/13 [==============================] - 3s 224ms/step - loss: 6.0197 - accura

KeyboardInterrupt: 

In [45]:

seed_text=lines[1234] 
seed_text 


'span classlabel stylescope mediabuttonwayback machinespan a mediabutton mediabutton classstylescope mediamenu xscope datamediatypetexts a classmenuitem texts stylescope mediabutton hrefhttpsarchiveorgdetailstexts dataeventclicktrackingbetatopnavnavmenutexts span classicon stylescope mediabutton svg arialabelledbytextstitleid textsdescid classstylescope mediabutton title idtextstitleid classstylescope mediabuttontexts icontitle desc idtextsdescid classstylescope mediabuttonan illustration of an open bookdesc path classfillcolor stylescope mediabutton svg span span classlabel stylescope'

In [53]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words): 
 text = [] 
 for _ in range(n_words): 
    encoded = tokenizer.texts_to_sequences([seed_text])[0] 
 encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre') 
 y_predict = model.predict_classes(encoded) 
 predicted_word = '' 
 for word, index in tokenizer.word_index.items(): 
    if index == y_predict: 
        predicted_word = word 
        break 
 seed_text = seed_text + ' ' + predicted_word 
 text.append(predicted_word) 
 return ' '.join(text) 


In [54]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100) 

C:\Users\Aditi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'mediabutton'